In [ ]:
from google.cloud import vision
from google.cloud.vision import types
from PIL import Image, ImageDraw

client = vision.ImageAnnotatorClient()
def detect_face(face_file, max_results=4):
    """Uses the Vision API to detect faces in the given file.

    Args:
        face_file: A file-like object containing an image with faces.

    Returns:
        An array of Face objects with information about the picture.
    """
    client = vision.ImageAnnotatorClient()

    content = face_file.read()
    image = types.Image(content=content)

    return client.face_detection(image=image).face_annotations
def highlight_faces(image, faces, output_filename):
    """Draws a polygon around the faces, then saves to output_filename.

    Args:
      image: a file containing the image with the faces.
      faces: a list of faces found in the file. This should be in the format
          returned by the Vision API.
      output_filename: the name of the image file to be created, where the
          faces have polygons drawn around them.
    """
    im = Image.open(image)
    draw = ImageDraw.Draw(im)

    for face in faces:
        box = [(vertex.x, vertex.y)
               for vertex in face.bounding_poly.vertices]
        draw.line(box + [box[0]], width=5, fill='#00ff00')

    im.save(output_filename)

In [ ]:
input_filename = "/home/topica/Pictures/Webcam/happy.jpg"
output_filename = "/home/topica/Pictures/Webcam/happy_result.jpg"
likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
  'LIKELY', 'VERY_LIKELY')
max_results = 3
with open(input_filename, 'rb') as image:
#     faces = detect_face(image, max_results)
#     print('Found {} face{}'.format(
#         len(faces), '' if len(faces) == 1 else 's'))
    resp = client.face_detection(image=image)
    faces = resp.face_annotations
    for face in faces:
        print(input_filename + ': surprise: {}'.format(likelihood_name[face.surprise_likelihood]))
    print('Writing to file {}'.format(output_filename))
    # Reset the file pointer, so we can read the file again
    image.seek(0)
    highlight_faces(image, faces, output_filename)

## Test GG API

In [1]:
import argparse
import io
import re
import numpy as np

# [START vision_face_detection]
def detect_faces(path):
    """Detects faces in an image."""
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    # [START vision_python_migration_face_detection]
    # [START vision_python_migration_image_file]
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)
    # [END vision_python_migration_image_file]

    response = client.face_detection(image=image)
    faces = response.face_annotations
    label = np.zeros(5)
#     labels = []
    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')
    print('Faces:')
    for face in faces:
        if likelihood_name[face.anger_likelihood] == 'UNKNOWN' or likelihood_name[face.anger_likelihood]=='VERY_UNLIKELY'or likelihood_name[face.anger_likelihood]=='UNLIKELY':
            label[0] = 0
        else:
            label[0] = 1
        if likelihood_name[face.joy_likelihood] == 'UNKNOWN' or  likelihood_name[face.joy_likelihood]=='VERY_UNLIKELY'or likelihood_name[face.joy_likelihood]=='UNLIKELY':
            label[1] = 0
        else:
            label[1] = 1
        if likelihood_name[face.sorrow_likelihood] == 'UNKNOWN' or  likelihood_name[face.sorrow_likelihood]=='VERY_UNLIKELY'or likelihood_name[face.sorrow_likelihood]=='UNLIKELY':
            label[2] = 0
        else:
            label[2] = 1
        if likelihood_name[face.surprise_likelihood] == 'UNKNOWN' or  likelihood_name[face.surprise_likelihood]=='VERY_UNLIKELY'or likelihood_name[face.surprise_likelihood]=='UNLIKELY':
            label[3] = 0
        else:
            label[3] = 1
        if sum(label) == 0:
            label [4] = 1
#         labels.append(label)
#         print(labels)
#         print(path)
#         print('anger: {}'.format(likelihood_name[face.anger_likelihood]))
#         print('joy: {}'.format(likelihood_name[face.joy_likelihood]))
#         print('surprise: {}'.format(likelihood_name[face.surprise_likelihood]))
#         print('sorrow: {}'.format(likelihood_name[face.sorrow_likelihood]))

#         vertices = (['({},{})'.format(vertex.x, vertex.y)
#                     for vertex in face.bounding_poly.vertices])

#         print('face bounds: {}'.format(','.join(vertices)))
    return label
    # [END vision_python_migration_face_detection]
# [END vision_face_detection]


In [2]:
emotions = ['anger', 'happy', 'sadness', 'surprise','neutral']

In [3]:
test_dir =  "/home/topica/workspace/data_ckp_crop"

In [4]:
import os
test_data =[]
test_label = []
for i in  range(len(emotions)):
    emotion_dir = os.path.join(test_dir,emotions[i])
    list_img = os.listdir(emotion_dir)
    print(list_img)
    for j in range(len(list_img)):
        img_dir = os.path.join(emotion_dir, list_img[j])
        test_data.append(img_dir)
        test_label.append(i)

['S133_003_00000047.png', 'S136_005_00000006.png', 'S503_001_00000039.png', 'S010_004_00000010.png', 'S501_001_00000041.png', 'S126_008_00000011.png', 'S029_001_00000015.png', 'S133_003_00000033.png', 'S066_005_00000011.png', 'S028_001_00000016.png', 'S501_001_00000021.png', 'S082_005_00000008.png', 'S127_010_00000018.png', 'S089_003_00000035.png', 'S133_003_00000024.png', 'S034_003_00000016.png', 'S022_005_00000014.png', 'S022_005_00000031.png', 'S109_003_00000010.png', 'S119_008_00000018.png', 'S503_001_00000057.png', 'S075_008_00000012.png', 'S999_001_00000016.png', 'S042_004_00000019.png', 'S503_001_00000066.png', 'S066_005_00000004.png', 'S029_001_00000018.png', 'S133_003_00000026.png', 'S028_001_00000018.png', 'S133_003_00000018.png', 'S067_004_00000015.png', 'S028_001_00000013.png', 'S050_004_00000021.png', 'S126_008_00000027.png', 'S112_005_00000011.png', 'S100_005_00000018.png', 'S067_004_00000018.png', 'S117_006_00000008.png', 'S037_003_00000010.png', 'S100_005_00000016.png',

In [5]:
test_data = np.asarray(test_data)
test_label = np.asarray(test_label)

In [6]:
test_data 

array(['/home/topica/workspace/data_ckp_crop/anger/S133_003_00000047.png',
       '/home/topica/workspace/data_ckp_crop/anger/S136_005_00000006.png',
       '/home/topica/workspace/data_ckp_crop/anger/S503_001_00000039.png',
       ...,
       '/home/topica/workspace/data_ckp_crop/neutral/S093_001_00000003.png',
       '/home/topica/workspace/data_ckp_crop/neutral/S011_001_00000002.png',
       '/home/topica/workspace/data_ckp_crop/neutral/S045_004_00000001.png'],
      dtype='|S67')

In [7]:
from sklearn.preprocessing import label_binarize
test_label = label_binarize(test_label, classes=[0, 1, 2, 3, 4])

In [ ]:
emotions = ['anger', 'joy', 'sorrow', 'surprise','neutral']

In [8]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/topica/MyFirstProject-542428fa5188.json"
print(os.environ['GOOGLE_APPLICATION_CREDENTIALS']) 

/home/topica/MyFirstProject-542428fa5188.json


In [ ]:
labels = []
for i in range(len(test_data)):
    label = detect_faces(test_data[i])
    labels.append(label)

Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:
Faces:

In [ ]:
labels = np.asarray(labels)

In [ ]:

import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc

# Plot linewidth.
lw = 2

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
n_classes = 5
roc_auc = dict()
y_score = labels
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(test_label[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(test_label.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue','gray','olive', 'cyan', ])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic to emotion detection on DATASET')
plt.legend(loc="lower right")
plt.show()
